# Grafting 2D MDS points onto **RELAXED SCAFFOLD** sphere

We have a scaffold projected as a sphere containing order points (130). Now, for each order, we need to take the 2D points (14x) and graft them onto the surface of the scaffold sphere, centered around each order point.

Basic method:
1. do 2D MDS plots for all orders
2. Graft these all onto the scaffold sphere
3. Place ones with too few points on at the same location as the order-level point (these have between 1 and 20 points)
4. Run spherical scatterplot relaxation
5. Re-compute order-level point based on centroid of relaxed points
6. re-run integrate_tree_to_xyz based on re-computed order-level points
7. Display everything

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

## The Scaffold

This is the initial positioning of the order points as determined from the order-level tree (130). They're separated relatively nicely and the tree branches make sense. This is a good initial condition, but we will move these around a bit as described above.

In [ ]:
# Load in the scaffold sphere points.

scaffold_filename = "output/random_taxa_mds_coords_norm_on_sphere.csv"

scaffold_df = pd.read_csv(scaffold_filename, index_col=0)

# These are normalized x,y, and z coordinates. Convert them to lat/lon.
scaffold_df["lat"] = np.degrees(np.arcsin(scaffold_df["z"]))
scaffold_df["lon"] = np.degrees(np.arctan2(scaffold_df["y"], scaffold_df["x"]))

# Plot the points on a sphere using plotly.
fig = px.scatter_geo(
    scaffold_df,
    lat="lat",
    lon="lon",
    hover_name=scaffold_df.index,
    title="Scaffold Sphere Order Points",
)
fig.update_geos(
    showcountries=False,
    showcoastlines=False,
    showland=False,
    showocean=False,
    showlakes=False,
    showrivers=False,
    projection_type="orthographic",
)
fig.update_layout(margin={"r":0,"t":30,"l":0,"b":0})
fig.update_layout(width=600, height=600)
fig.show()

## Graft order points from 2D projection (MDS) onto the sphere

Let's see if this works for a single order, Siluriformes


In [ ]:
# Load in the mMDS results for the current order.

import pathlib

current_order = 'Siluriformes'  # Change this to visualize a different order

mds_by_order_output_dir = pathlib.Path('output/mds_by_order')
    
df_mds = pd.read_csv(mds_by_order_output_dir / f"{current_order}_2D_mMDS_sklearn.csv", index_col=0)

# Normalize these points to be between -1 and 1 in both dimensions.
df_mds['x'] = (df_mds['x'] - df_mds['x'].min()) / (df_mds['x'].max() - df_mds['x'].min()) * 2 - 1
df_mds['y'] = (df_mds['y'] - df_mds['y'].min()) / (df_mds['y'].max() - df_mds['y'].min()) * 2 - 1
# Scale them down quite a bit to fit better on the sphere.
df_mds['x'] *= 0.3
df_mds['y'] *= 0.3

# Scale the X up a bit more to account for the sphere's aspect ratio.
df_mds['x'] *= 1.5

# Find the lat/lon of the center point for this order from the scaffold points.
center_point = scaffold_df.loc[current_order]
center_lat = center_point["lat"]
center_lon = center_point["lon"]

# Convert the normalized MDS points to lat/lon around the center point.
def mds_to_lat_lon(x, y, center_lat, center_lon):
    # Convert center lat/lon to radians
    center_lat_rad = np.radians(center_lat)
    center_lon_rad = np.radians(center_lon)

    # Calculate the new latitude
    new_lat_rad = np.arcsin(np.sin(center_lat_rad) * np.cos(np.sqrt(x**2 + y**2)) +
                            np.cos(center_lat_rad) * np.sin(np.sqrt(x**2 + y**2)) * (y / np.sqrt(x**2 + y**2)))

    # Calculate the new longitude
    new_lon_rad = center_lon_rad + np.arctan2(x * np.sin(np.sqrt(x**2 + y**2)),
                                              np.cos(np.sqrt(x**2 + y**2)) - np.sin(center_lat_rad) * np.sin(new_lat_rad))

    # Convert back to degrees
    new_lat = np.degrees(new_lat_rad)
    new_lon = np.degrees(new_lon_rad)

    return new_lat, new_lon
df_mds[['lat', 'lon']] = df_mds.apply(lambda row: mds_to_lat_lon(row['x'], row['y'], center_lat, center_lon), axis=1, result_type='expand')

# Plot the MDS points and the scaffold points on the same sphere.
fig = px.scatter_geo(scaffold_df,lat="lat",lon="lon",hover_name=scaffold_df.index,title=f"MDS Points for {current_order} on Scaffold Sphere",)
fig.add_scattergeo( lat=df_mds['lat'],lon=df_mds['lon'],hovertext=df_mds.index,mode='markers',marker=dict(size=5, color='red'),name='MDS Points')
fig.update_geos(showcountries=False,showcoastlines=False,showland=False,showocean=False,showlakes=False,showrivers=False,projection_type="orthographic")
fig.update_layout(margin={"r":0,"t":30,"l":0,"b":0})
fig.update_layout(width=600, height=600)
fig.show()

In [ ]:
# Now convert the lat/lon back to x,y,z coordinates on the unit sphere.
def lat_lon_to_xyz(lat, lon):
    lat_rad = np.radians(lat)
    lon_rad = np.radians(lon)
    x = np.cos(lat_rad) * np.cos(lon_rad)
    y = np.cos(lat_rad) * np.sin(lon_rad)
    z = np.sin(lat_rad)
    return x, y, z
df_mds[['x', 'y', 'z']] = df_mds.apply(lambda row: lat_lon_to_xyz(row['lat'], row['lon']), axis=1, result_type='expand')

df_mds.to_csv(mds_by_order_output_dir / f"{current_order}_3D_mMDS_on_sphere.csv")

In [ ]:
# Load these xyz points back in and plot them in 3D to verify.
df_3d = pd.read_csv(mds_by_order_output_dir / f"{current_order}_3D_mMDS_on_sphere.csv", index_col=0)
fig = px.scatter_3d(df_3d, x='x', y='y', z='z', color='order', title=f"3D mMDS of {current_order} on Sphere")
fig.update_layout(width=600, height=600)
fig.show()

## Grafting of all orders

Now we do what we did above, but for every order.

There are some exceptions - if the number of points is very small, move them all to be the same as the order-level initial condition point. They will be moved away from each other during the spherical relaxation step.

In [ ]:
# Get all the orders from the scaffold points.
orders = scaffold_df.index.tolist()

for current_order in orders:
    # Load in the mMDS results for the current order. If the file doesn't exist, we need to handle it
    # differently.
    df_mds  = None
    print(f"Processing order: {current_order}...", end='', flush=True)
    infile = f"output/{current_order}_2D_tSNE_sklearn.csv"
    # If the t-SNE file doesn't exist, try the mMDS file.
    if not pd.io.common.file_exists(infile):
        infile = f"output/{current_order}_2D_mMDS_sklearn.csv"
    df_mds = pd.read_csv(infile, index_col=0)

    # Print the number of genera in this order.
    print(f" {len(df_tsne)} genera...", end='', flush=True)

    # Standard scaling factor for most orders. If the number of genera is small, we use a smaller
    # scaling factor to keep them from being too spread out on the sphere.
    scaling_factor = 0.3
    if len(df_tsne) <= 500 and len(df_tsne) > 10:
        scaling_factor = 0.15
    elif len(df_tsne) <= 10:
        scaling_factor = 0.01

    print(f" scaling factor {scaling_factor}...", end='', flush=True)

    # Normalize these points to be between -1 and 1 on both axes.
    df_tsne['x'] = (df_tsne['x'] - df_tsne['x'].min()) / (df_tsne['x'].max() - df_tsne['x'].min()) * 2 - 1
    df_tsne['y'] = (df_tsne['y'] - df_tsne['y'].min()) / (df_tsne['y'].max() - df_tsne['y'].min()) * 2 - 1

    # Scale them down quite a bit to fit better on the sphere.
    df_tsne['x'] *= scaling_factor
    df_tsne['y'] *= scaling_factor

    # Scale the X up a bit more to account for the sphere's aspect ratio.
    df_tsne['x'] *= 1.5

    # Find the lat/lon of the center point for this order from the scaffold points.
    center_point = scaffold_df.loc[current_order]
    center_lat = center_point["lat"]
    center_lon = center_point["lon"]

    # Convert the normalized t-SNE points to lat/lon around the center point.
    # Note that this relies on tsne_to_lat_lon being defined above.
    df_tsne[['lat', 'lon']] = df_tsne.apply(lambda row: tsne_to_lat_lon(row['x'], row['y'], center_lat, center_lon), axis=1, result_type='expand')

    # Convert the lat/lon to x,y,z coordinates on the unit sphere. Note that
    # this relies on lat_lon_to_xyz being defined above.
    df_tsne[['x', 'y', 'z']] = df_tsne.apply(lambda row: lat_lon_to_xyz(row['lat'], row['lon']), axis=1, result_type='expand')
    df_tsne.to_csv(f"output/{current_order}_3D_on_sphere.csv")
    print("done.")

In [ ]:
# Merge all the orders into one big file.
orders_to_merge = orders
merged_df = pd.DataFrame()
for current_order in orders_to_merge:
    df_3d = pd.read_csv(f"output/{current_order}_3D_on_sphere.csv", index_col=0)
    merged_df = pd.concat([merged_df, df_3d], axis=0)
merged_df.to_csv("output/Actinopterygii_all_orders_3D_on_sphere.csv")

## Viz and relaxation of all orders

Now we run relaxation to better spread the points out on the sphere. This uses Wandrille's `spherical-scatterplot-relaxation` script.

In [ ]:
# Load in the one big file and plot it in 3D, colored by order.
df_3d = pd.read_csv("output/Actinopterygii_all_orders_3D_on_sphere.csv", index_col=0)
fig = px.scatter_3d(df_3d, x='x', y='y', z='z', color='order', title="3D t-SNE of All Actinopterygii on Sphere")
fig.update_layout(width=800, height=800)
fig.show()


mkdir -p odonata_family_tentative_distances_from_bybee2021.spherical_SMACOF.relax


python /mnt/e/git/spherical-scatterplot-relaxation/spherical_scatterplot_relaxation.py -i odonata_family_tentative_distances_from_bybee2021.spherical_SMACOF.csv \
      -o odonata_family_tentative_distances_from_bybee2021.spherical_SMACOF.relax/odonata_family_sphere \
      -n 100 -w 5 -l 0.05 



In [ ]:
# Run the scatterplot relaction script on each of the big four orders to better spread out the points on the sphere.
import os
os.makedirs("output/Actinopterygii_tree_order_DR_relaxed", exist_ok=True)

%run ./spherical-scatterplot-relaxation/spherical_scatterplot_relaxation.py -i output/Actinopterygii_all_orders_3D_on_sphere.csv -o output/Actinopterygii_tree_order_DR_relaxed/Actinopterygii_all_orders_3D_on_sphere_DR_relaxed.csv -n 500 -n 100 -w 5 -l 0.05

In [ ]:
# Use a slider to visualize the relaxation process.
fig = go.Figure()
rounds_to_plot = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100]
# Add traces, one for each slider step
for step in range(len(rounds_to_plot)):
    filename = f"output/Actinopterygii_tree_order_DR_relaxed/Actinopterygii_all_orders_3D_on_sphere_DR_relaxed.csv_round{rounds_to_plot[step]}.csv"

    # reading data for that round
    tmp = pd.read_csv( filename , index_col= 0 )

    fig.add_trace(
        go.Scatter3d(
            x=tmp['x'],
            y=tmp['y'],
            z=tmp['z'],
            mode='markers',
            marker=dict(size=3, color=tmp['order'].astype('category').cat.codes, colorscale='Viridis', opacity=0.8),
            hovertext=tmp.order,
            name=f'Round {rounds_to_plot[step]}',
            visible=(step == 0)  # Only the first trace is visible initially
        )
    )
# Create slider steps
steps = []
for i in range(len(rounds_to_plot)):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(rounds_to_plot)},
              {"title": f"3D t-SNE of All Actinopterygii on Sphere - Round {rounds_to_plot[i]}"}],  # layout attribute
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)
sliders = [dict(
    active=0,
    currentvalue={"prefix": "Round: "},
    pad={"t": 50},
    steps=steps
)]
fig.update_layout(
    sliders=sliders,
    width=800,
    height=800,
    title="3D t-SNE of All Actinopterygii on Sphere"
)
fig.show()

Round 55 looks pretty good... now to import into OpenSpace...

In [ ]:
# Quick check. How many unique families total?
df_final = pd.read_csv("output/Actinopterygii_tree_order_DR_relaxed/Actinopterygii_all_orders_3D_on_sphere_DR_relaxed.csv_round55.csv", index_col=0)
print(f"Total genera: {len(df_final)}")
# Count number of unique families
unique_families = df_final['family'].nunique()
print(f"Total families: {unique_families}")

In [ ]:
# Find points with NaN x, y, or z values.
nan_points = df_final[df_final[['x', 'y', 'z']].isnull().any(axis=1)]
if len(nan_points) > 0:
    print("Warning: Found points with NaN x, y, or z values in the dataset.")
    print(nan_points)

How many duplicate points? This should match what OpenSpace reports.

In [ ]:
duplicate_xyz = df_final[df_final.duplicated(subset=["x", "y", "z"], keep=False)]
if len(duplicate_xyz) > 0:
    print("Warning: Found duplicate x, y, z values in the dataset.")
    print(duplicate_xyz)

## Re-making scaffold points

The original scaffold points were used to provide initial positions for the order-level points. We've moved these around, so now we need to re-compute the shifted positions. To do this, on an order-by-order basis, we'll compute the centroid and use that as the location of the order-level point. We'll then export the new scaffold to run through Wandrille's `integrate_tree_to_xyz` script.


In [ ]:
# scaffold_df is the original scaffold points dataframe. Let's steal the
# Zoraptera point for the new one.

actinopterygii_sphere_points = pd.read_csv("output/Actinopterygii_tree_order_DR_relaxed/Actinopterygii_all_orders_3D_on_sphere_DR_relaxed.csv_round55.csv", index_col=0)

# Columns for the new df are order, x, y, z.
new_scaffold_df = pd.DataFrame(columns=['order', 'x', 'y', 'z'])

for order in orders:
    if order in actinopterygii_sphere_points['order'].values:
        subset = actinopterygii_sphere_points[actinopterygii_sphere_points['order'] == order]
        centroid = subset[['x', 'y', 'z']].mean()
        # What's the radius of this centroid?
        radius = np.sqrt(centroid['x']**2 + centroid['y']**2 + centroid['z']**2)
        # Normalize the centroid to be on the unit sphere.
        centroid['x'] /= radius
        centroid['y'] /= radius
        centroid['z'] /= radius
        # print this out.
        new_scaffold_df.loc[len(new_scaffold_df)] = {'order': order, 'x': centroid['x'], 'y': centroid['y'], 'z': centroid['z']}
    else:
        print(f"Order {order} not found in points, skipping.")

# Amiiformes, Spariformes, and Centrarchiformes are missing from the t-SNE points, so we'll just use the original scaffold points for those.
for order in ['Amiiformes', 'Spariformes', 'Centrarchiformes']:
    # Get the index of this order in the new scaffold.
    index = new_scaffold_df[new_scaffold_df['order'] == order].index[0]

    # Get the original point for this order.
    (x, y, z) = scaffold_df.loc[order, ['x', 'y', 'z']]

    new_scaffold_df.loc[index] = {'order': order, 'x': x, 'y': y, 'z': z}

new_scaffold_df.to_csv("output/Actinopterygii_tree_order_DR_relaxed/Actinopterygii_tree_order_DR_relaxed_scaffold.csv", index=False)

## The Tree

Finally, Wandrille's tree script to make branches.

In [ ]:
%run ./integrate_tree_to_XYZ/integrate_tree_to_XYZ.py -i "output/Actinopterygii_tree_order_DR_relaxed/Actinopterygii_tree_order_DR_relaxed_scaffold.csv" -t "output/Actinopterygii_order_level.nwk" -o "output/Actinopterygii_tree_order_DR_relaxed/Actinopterygii_tree_order_DR_relaxed_scaffold" --ignore-missing --use-z-from-file